# This is the third part to assignment for week3--its really challenging !

In [5]:
import numpy as np
import pandas as pd
import geopy 
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import folium
import requests
from tqdm import tqdm
from collections import deque
import matplotlib.cm as cm
import matplotlib.colors as colors

## Load previous data

In [36]:
# loading data generated in previous file
finalData = pd.read_csv("myCombined_data.csv")
finalData.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [40]:
print('Now we have {} boroughs and {} neighborhoods.'.format(
        len(finalData['Borough'].unique()),
        finalData.shape[0]
    )
)

Now we have 10 boroughs and 103 neighborhoods.


In [38]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
# Map Visulization with geopy
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of city Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of city Toronto are 43.653963, -79.387207.


In [44]:
# create map of New York using latitude and longitude values
mapToronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = finalData

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                           neighborhoods['Longitude'],
                                           neighborhoods['Borough'],
                                           neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(mapToronto)  
    
mapToronto

## Configure Foursquare access

In [46]:
# The code was removed by Watson Studio for sharing.

Your credentails:


Coustom function that takes the names and locations of the neighborhoods in Toronto and obtains the 100 top venues around.

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in tqdm(zip(names, latitudes, longitudes), total = names.size):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
Toronto_venues = getNearbyVenues(TorontoData.Neighbourhood,
                            TorontoData.Latitude,
                            TorontoData.Longitude)

100%|██████████| 103/103 [01:17<00:00,  1.33it/s]


getting info for check

In [15]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2223, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Morningside, Guildwood, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"Morningside, Guildwood, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,"Morningside, Guildwood, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


 Also check the amount of venues per neighboorhood

In [16]:
Toronto_venues.groupby("Neighborhood").Venue.count().sort_values(ascending=False).head()

Neighborhood
Adelaide, King, Richmond                  100
Victoria Hotel, Commerce Court            100
Ryerson, Garden District                  100
St. James Town                            100
Underground city, First Canadian Place    100
Name: Venue, dtype: int64

In [17]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


## One Hot enconding of these data.

In [47]:
Toronto_encoded = pd.get_dummies(Toronto_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_encoded["Neighborhood"] = Toronto_venues["Neighborhood"]


nindex = list(Toronto_encoded.columns).index("Neighborhood")
cols = deque(Toronto_encoded.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_encoded = Toronto_encoded[cols]

Toronto_encoded.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,...,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Port Union, Rouge Hill, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Morningside, Guildwood, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Morningside, Guildwood, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Morningside, Guildwood, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
Toronto_encoded.shape

(2223, 268)

In [49]:
#Find average number of venue categories per neighborhood
Toronto_grouped = Toronto_encoded.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,...,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue
0,"Adelaide, King, Richmond",0.01,0.0,0.01,0.01,0.01,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,"Agincourt North, Milliken, Steeles East, L'Amo...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,"Bathurst Manor, Downsview North, Wilson Heights",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.05,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,Bayview Village,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [50]:
#I found 99 Neighborhoods, grouped by borough, and 268 distinct categories
Toronto_grouped.shape

(99, 268)


it may be that for some postcodes there are no venues


In [22]:
# let us select only the N most frequent venue categories per neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Breakfast Spot,Sushi Restaurant,American Restaurant,Asian Restaurant,Gym
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Arts & Crafts Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Music Venue
2,"Agincourt North, Milliken, Steeles East, L'Amo...",Playground,Park,Music Venue,Airport Terminal,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports
3,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Chinese Restaurant,Shopping Mall,Supermarket,Sushi Restaurant,Sandwich Place,Deli / Bodega,Pizza Place,Pharmacy,Restaurant
4,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Warehouse Store,Wine Bar,Bakery,Bagel Shop,Video Store,Baby Store



## Neighborhood clustering

Before clustering, let us use a PCA decomposition to reduce the noise of the signal and improve clustering efficiency


In [53]:
pca = PCA(.90)
Toronto_grouped_clustering = pca.fit_transform(Toronto_grouped.drop('Neighborhood', 1))
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

In [54]:
#The dimensionality reduction has kept only a reduced number of the total dimensions
Toronto_grouped_clustering.shape

(99, 267)

In [55]:
#With reduced dimensional dataset we carry out the KMeans clustering
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[3 3 0 3 3 3 3 0 3 0]
(99,)


Try the location and the cluster information, together with the top 10 venues

In [56]:
Toronto_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
Toronto_combined = TorontoData.merge(Toronto_grouped, left_on = "Neighbourhood", right_on = "Neighborhood", how = "outer")

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_combined = Toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_combined["Cluster Labels"] = Toronto_combined["Cluster Labels"].fillna(5).astype("int")

Toronto_combined.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,"Rouge, Malvern",0.0,0.0,0.0,0.0,...,Fast Food Restaurant,Music Venue,Airport Terminal,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,"Port Union, Rouge Hill, Highland Creek",0.0,0.0,0.0,0.0,...,Bar,Airport Terminal,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711,"Morningside, Guildwood, West Hill",0.0,0.0,0.0,0.0,...,Mexican Restaurant,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Medical Center,Spa,Antique Shop,Aquarium,Art Gallery
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Woburn,0.0,0.0,0.0,0.0,...,Coffee Shop,Insurance Office,Korean Restaurant,Athletics & Sports,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Cedarbrae,0.0,0.0,0.0,0.0,...,Caribbean Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Bank,Gas Station,BBQ Joint,Auto Garage


In [57]:
# now create map
mapClusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_combined['Latitude'],
                                  Toronto_combined['Longitude'],
                                  Toronto_combined['Neighborhood'],
                                  Toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapClusters)
       
mapClusters

## Analyze clusters
### Cluster 1

#### Cluster 1 contains many restaurants and diners, as well as some other services, like bus and train stations


In [58]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 0, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Playground,Park,Music Venue,Airport Terminal,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports
23,Convenience Store,Bank,Park,Athletics & Sports,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Music Venue
25,Food & Drink Shop,Park,Music Venue,Airport Terminal,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports
30,Park,Airport,Snack Place,Music Venue,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage
31,Grocery Store,Hotel,Park,Bank,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports,Baby Store,Airport Terminal



## Cluster 2

#### Cluster 2 seems to be dominated by banks, music venues and airport facilities


In [59]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 1, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Baseball Field,Airport Terminal,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports
97,Baseball Field,Airport Terminal,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports



## Cluster 3

#### Cluster 3 is mostly linked to open areas for sport and children


In [60]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 2, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Restaurant,Auto Garage,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Music Venue,Baseball Field


## Cluster 4

#### Cluster 4 is the most homogeneous one (it only contains two neighborhoods) and seems to be dominated by the Baseball field


In [61]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 3, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fast Food Restaurant,Music Venue,Airport Terminal,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports
1,Bar,Airport Terminal,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports
2,Mexican Restaurant,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Medical Center,Spa,Antique Shop,Aquarium,Art Gallery
3,Coffee Shop,Insurance Office,Korean Restaurant,Athletics & Sports,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage
4,Caribbean Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Bank,Gas Station,BBQ Joint,Auto Garage



## Cluster 5

### Cluster 5 is characterized by shops and restaurants


In [62]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 4, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Playground,Music Venue,Athletics & Sports,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,Airport Terminal


## and that's All!